<a href="https://colab.research.google.com/github/nagaokayama/captioning_scripts/blob/main/Text_cordinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. 🌄google driveのファイルを使用する場合はまずこのスクリプトを実行してください {display-mode: "form"}
# @markdown ←●に▶が入ってるマークが「セルを実行ボタン」です。これを押して実行\
# @markdown グーグルドライブの中身はファイルブラウザのgdrive/MyDriveに共有されておりここから閲覧できます


# @markdown 左端にあるフォルダのマークをクリックしてファイルブラウザを開き\
# @markdown 目的のフォルダにマウスオーバーすると表示される縦に3点が並んだマークをクリック→「パスをコピー」でファイルパスをコピーできます
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

# @title 0.🎍必要なモジュールのインポートと関数の設定　{display-mode: "form"}
# @markdown ステップ1: セルを実行\
# @markdown このノートブックを開いたらまず始めにこのセルを一度だけ実行してください\
# @markdown 下準備のためのセルなのでほかのセルを実行するたびにこのセルを実行しなおす必要はありません。
# name = "" # @param type{type:"string"}

!pip install pyheif

import os
import csv
import re
import glob
import chardet
import math
import shutil
from PIL import Image, ExifTags
import pyheif


def find_image_files(imagesFolder,exportFullpath = False):
    # 引数にいれたフォルダの中から画像ファイルを探してファイルのリストを返す
    # globで簡単に全部のファイルをリストした後にre.searchで画像ファイルだけを抽出する

    fileNameList = []

    tmpFiles = glob.glob(f"{imagesFolder}/*.*")
    tmpFiles.sort()

    if len(tmpFiles) == 0:
        return -1

    for name in tmpFiles:
        # 画像ファイル拡張子を持つファイルを抽出,大文字小文字を区別しない
        pattern = re.compile(".*\.(png|jpg|jpeg|gif|webp|tiff|heic)$",re.IGNORECASE)
        basename = os.path.basename(name)

        ismatch = pattern.search(basename)
        if ismatch:
            if exportFullpath:
                tmp = name
            else:
                tmp = ismatch.group(0).split(".")[0] # 拡張子を削除
            fileNameList.append(tmp)

    if len(fileNameList) == 0:
        return []

    return fileNameList

def detect_encoding(file_path):
    # エンコードの自動判定 100％の精度ではないらしい

    with open(file_path, 'rb') as file:
        rawdata = file.read()
    result = chardet.detect(rawdata)
    return result['encoding']

def create_zip(sourcePath_list , zipPath):
    print("zip作成")

    for folder_i_want_archive in sourcePath_list:
        zipname = folder_i_want_archive.split(r"/")[-1]
        export_path = os.path.join(zipPath, zipname)
        shutil.make_archive(export_path,"zip",folder_i_want_archive)
        print(export_path+".zip")



In [ ]:
# @title 🎊ZIPファイルを展開　{display-mode: "form"}
# @markdown zipファイルを指定フォルダに解凍します

# @markdown ファイルブラウザで目的のフォルダにマウスオーバーすると縦に3点が並んだマークが表示されます。\
# @markdown　3点マークをクリック→「パスをコピー」でファイルパスをコピーできます
import_zip_path = ''  # @param {type:"string"}
export_path = ''  # @param {type:"string"}

try:
    shutil.unpack_archive(import_zip_path, export_path)
    print("解凍しました")
except Exception as e:
    print(e)

In [ ]:

# 共通キャプションを持ったcsvファイルを生成
# UIはタイプスクリプトで書いてるみたい

# @title 📖画像名と共通キャプションを各行に書き込んだcsvを生成　{display-mode: "form"}
# @markdown ステップ1:画像ファイルのフォルダを指定
image_folder_path = ''  # @param {type:"string"}

# @markdown ステップ2: すべての画像に書き込むキャプションをカンマor読点区切りで入力\
# @markdown 共通キャプションがない場合は空欄にしてください
# unicodeでそのまま書かれちゃうけどrを入れてなければちゃんと解釈されるっぽい？
common_caption = '' # @param {type:"string"}

# @markdown ステップ3: CSVファイルの書き出し先を指定
# CSVファイルにデータを書き込み
csv_file_path = ''  # @param {type:"string"}

# @markdown ## オプション
# @markdown microsoft エクセルでCSVファイルを読み込む場合はチェックをつける（shift-jisでcsvが生成されます）
export_with_shiftjis = False # @param {type:"boolean"}

# @markdown 必要事項が書けたらセルを実行

def generate_csv_caption(image_files,common_captions="", encoding = "utf8" ):
    common_caption_words = re.split(r',|、', common_captions)
    data = []

    for image_file in image_files:
        # 画像名と共通キャプションをまとめてcsv用の配列を生成
        data.append([image_file] + common_caption_words)

    data.sort()

    csv_fullpath = os.path.join(csv_file_path, 'imageAndCaptions_data.csv')
    with open(csv_fullpath, 'w', newline='', encoding = encoding) as csv_file:
        csv_writer = csv.writer(csv_file)

        # データを書き込む
        csv_writer.writerows(data)

    print(f"CSVファイルが {csv_fullpath} に生成されました。")




imageList = find_image_files(image_folder_path)

if imageList == -1:
    print(f"エラー: {image_folder_path} に画像ファイルが存在しません")
else:
    # HACK:トグルでエンコードを切り替えたい。こいつはあんまりきれいじゃないと思う
    if export_with_shiftjis :
        encode_mode = "shift-jis"
    else:
        encode_mode = "utf8"
    generate_csv_caption(imageList,common_caption,encode_mode)



In [ ]:

# @title ✒csvファイルから各画像用キャプションテキストファイルを生成　{display-mode: "form"}

# @markdown ステップ1:  csvファイルを指定
csv_file_path = '' # @param {type:"string"}

# markdown ステップ1.オプション:  画像フォルダを指定
images_folder_path = '/content/imageAndCaptions_data.csv' # param {type:"string"}

# csv_file_path = '/content/Untitled Folder/image_data.csv' # param type{"string"}
# @markdown ステップ2: キャプションファイルの書き出しフォルダを指定
text_folder_path = ''  # @param {type:"string"}

# @markdown 「セルを実行」ボタンを押してキャプションテキストファイルを生成


def getFilenamesAndTagsFromCsv(csv_file_path=csv_file_path):

    encoding = detect_encoding(csv_file_path)
    print(f"The detected encoding is: {encoding}")

    with open(csv_file_path, 'r', newline='', encoding=encoding) as csv_file:
        csv_reader = csv.reader(csv_file)
        captionsList = []
        imagenameList = []
        # rowは縦だっけ？
        for row in csv_reader:
            # 各行の要素について処理
            imagenameList.append(row[0])
            # row[0]以外のすべての要素をタグとして配列に入れる
            # 空の要素をテキストファイルに入れないように排除
            tmp = [i for i in row[1:] if i != ""]
            captionsList.append(tmp)

    return imagenameList, captionsList


def generateTextFileFromList(imagenameList, captionsList):

    for n, imagename in enumerate(imagenameList):
        # captionsのためにわざわざenumerate()つけるのなんかやだかも
        captions = captionsList[n]
        text_file_path = f'{text_folder_path}/{imagename}.txt'

        concat_captions = "、".join(captions)
        with open(text_file_path, 'w') as text_file:
            text_file.write(concat_captions)

    print(f"キャプションファイルを {text_folder_path}に保存済み")


imagenameList, captionsList = getFilenamesAndTagsFromCsv(csv_file_path)
generateTextFileFromList(imagenameList, captionsList)

In [ ]:
# @title 🎨学習に適したサイズに画像をリサイズ（かなり自己責任でお願いします）　{display-mode: "form"}

# @markdown 指定したフォルダに入ってる画像を約110万画素（2の20乗を少し超えるくらい）になるようにリサイズします。\
# @markdown リサイズの結果短辺が512pxを下回る場合は短辺が512pxになるように再計算されます。\
# @markdown heicファイルは処理の都合でリサイズ後にjpegへ返還されます\
# @markdown なおリサイズの結果画質が下がる可能性があります、\
# @markdown また安全のため必ず画像のバックアップを取ってから使用してください。

# @markdown ---

# @markdown 画像のフォルダ
image_dir_name = '' # @param {type:"string"}
# @markdown リサイズ後の画像の保存先を指定
new_dir_name = '' # @param {type:"string"}
# @markdown 「セルを実行」ボタンを押して画像をリサイズ


def heic2img(file):
    heic_path = file

    # HEICファイルをPNGに変換
    heif_file = pyheif.read(heic_path)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )

    return image

# この関数を追加
def rotate_image_based_on_exif(file_path):
    try:
        with Image.open(file_path) as img:
            for orientation in ExifTags.TAGS.keys():
                if ExifTags.TAGS[orientation] == 'Orientation':
                    break
            exif = dict(img._getexif().items())

            if exif[orientation] == 3:
                img = img.rotate(180, expand=True)
            elif exif[orientation] == 6:
                img = img.rotate(270, expand=True)
            elif exif[orientation] == 8:
                img = img.rotate(90, expand=True)

        img.save(file_path)
    except (AttributeError, KeyError, IndexError):
        # 例外処理が必要に応じて拡充する
        print("error")
        pass

def extract_fullfileName(imagesFolder):
    # 引数にいれたフォルダの中から画像ファイルを探してファイルのリストを返す
    # globで簡単に全部のファイルをリストした後にre.searchで画像ファイルだけを抽出する

    fileNameList = []

    tmpFiles = glob.glob(f"{imagesFolder}/*.*")
    tmpFiles.sort()

    if len(tmpFiles) == 0:
        print("ファイルが存在しません")
        return -1

    for name in tmpFiles:
        # 画像ファイル以外の拡張子を持つファイルを除外
        pattern = re.compile(".*\.(png|jpg|jpeg|gif|webp|tiff)$",re.IGNORECASE)
        # basename = os.path.basename(name)

        ismatch = pattern.search(name)
        if ismatch:
            tmp = ismatch.group()
            fileNameList.append(tmp)

    if len(fileNameList) == 0:
        print("画像ファイルが存在しません")
        return -1

    return fileNameList


IMAGE_RESOLUSION = 1100000 # 2^20画素が学習に十分な数

def resize_image_by_reso(image, resolusion:int):
    img = image
    image_ratio = img.height / img.width

    resizeH = int(math.sqrt(image_ratio * resolusion)) # たぶん高さが出る
    resizeW = resolusion // resizeH

    if resizeH <= 512 or resizeW <= 512:
        # 画像サイズは短辺512以上が必須
        fix_ratio = max((512 / resizeH),(512 / resizeW))
        resizeH = int(resizeH * fix_ratio)
        resizeW = int(resizeW * fix_ratio)

        # print(f"width{resizeW}, height{resizeH}")

    # サイズは（幅、高さ）
    img_resized = img.resize((resizeW, resizeH))

    return img_resized


if image_dir_name == new_dir_name:
    print("あぶない：画像のフォルダとリサイズ後の画像の保存先が同一")

else:
    output_path  = new_dir_name
    if os.path.exists(output_path):
        pass
    else:
        os.makedirs(output_path)
    files = find_image_files(image_dir_name,True)

    for file in files:
        # rotate_image_based_on_exif(file)
        # 力技の分岐

        if file.endswith(".heic") or file.endswith(".HEIC"):
            target_image = heic2img(file)
            target_image = resize_image_by_reso(target_image,IMAGE_RESOLUSION)

            new_file_name = os.path.splitext(os.path.basename(file))[0] + ".jpeg"
            target_image.save(os.path.join(output_path,new_file_name), 'JPEG')
        else:
            target_image = Image.open(file)
            target_image = resize_image_by_reso(target_image,IMAGE_RESOLUSION)
            target_image.save(os.path.join(output_path, os.path.basename(file)))
    print("リサイズ済")



In [ ]:

# @title 👾画像（とキャプションtxt）を100個ずつフォルダに分配する（ちょっと慎重に使ってね）　{display-mode: "form"}
# @markdown  フォルダ内の画像を100枚ずつ別フォルダへ分配します。\
# @markdown 分配に指定したフォルダの下位にサブフォルダが作成され、ここに100枚ずつ画像がコピーされます。\
# @markdown 画像と同名のテキストファイル（キャプションファイル）がある場合はこれも同時に分配されます。


# @markdown ---

# @markdown ステップ1. 分配したい画像が入っているフォルダを指定
image_folder = "" #@param {type:"string"}
# @markdown ステップ2. 分配先のフォルダを指定
destination_export_folder = "" #@param {type:"string"}

# @markdown ## オプション
# @markdown チェックをつけると画像がコピーされることなく直接サブフォルダに分配(移動)され、1.で指定した画像が入っているフォルダは空になります
move_orignalFiles = False #@param {type:"boolean"}

# @markdown サブフォルダごとにzipファイルを作成します。zipファイルは1.で指定したフォルダに作成されます。\
# @markdown ※discordサーバーにアップロード可能なファイルサイズを考慮しません
pack_into_zip = False #@param {type:"boolean"}

# @markdown 「セルを実行」ボタンを押してファイルを分配

def create_subfolders(rootFolder, numberOfSubfolders):
    result = []
    for i in range(numberOfSubfolders):
        new_dir_path = os.path.join(rootFolder, f'sub_{i+1}')
        result.append(new_dir_path)
        if os.path.exists(new_dir_path) is False:
            os.makedirs(new_dir_path)
    return result

def file_distribute_in_subfolder():
    # 100ファイルずつサブフォルダーに移動させる
    # TODO：ちゃんと関数に引数をとりたい
    imagenameList = find_image_files(imagesFolder=image_folder,exportFullpath=True)

    if len(imagenameList) >1:
        numberOfSubfolders = math.ceil(len(imagenameList) / 100)
        subfolderFullpath_list = create_subfolders(destination_export_folder, numberOfSubfolders)

    else:
        print("分配未実行：画像が100枚未満")
        return

# 関数名を適当に書くな
    imageNames_classified_bysubFolder = []
    for i in range(numberOfSubfolders):
        imageNames_classified_bysubFolder.append(imagenameList[0:100])
        del imagenameList[0:100]


    for n, imageNames_in_Subfolder in enumerate(imageNames_classified_bysubFolder):
        destination = subfolderFullpath_list[n]
        for imageNameFullpath in imageNames_in_Subfolder:

            pattern = re.compile('\\.[a-zA-Z]{3,4}$')

            textNameFullpath = pattern.sub('.txt', imageNameFullpath)
            # HACK:テキストファイルはあってもなくてもいい、という仕様を適当に実装した
            if os.path.exists(textNameFullpath):
                # print(textNameFullpath)
                if move_orignalFiles :
                    shutil.move(textNameFullpath, destination)
                else :

                    shutil.copy2(textNameFullpath, destination)
            else:
                pass
                print(f"ないよ {textNameFullpath}")


            # print(imageNameFullpath)
            if move_orignalFiles :
                shutil.move(imageNameFullpath, destination)
            else :
                shutil.copy2(imageNameFullpath, destination)


    if pack_into_zip:
        create_zip(subfolderFullpath_list)

    print ('done!')
    return


file_distribute_in_subfolder()



In [ ]:
# @title 📦ZIPファイルを作成
# @markdown  指定したフォルダの中身をzipに圧縮します（複数フォルダ可）\
# @markdown  複数フォルダを一度に圧縮したいときはフォルダパスを,か、で区切って記入してください

source_folder = "" #@param {type:"string"}
source_folder = re.split("[,、]",source_folder)

export_folder = "" #@param {type:"string"}

create_zip(source_folder, export_folder)

In [ ]:

# @title 各画像ファイル名をテキストファイルに書き込む {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

# @markdown 左端にあるフォルダのマークをクリックしてファイルブラウザを開き\
# @markdown  目的のフォルダにマウスオーバーすると表示される縦に3点が並んだマークをクリック→「パスをコピー」でファイルパスをコピーできます

# @markdown 画像の入っているフォルダを指定
image_folder = "" #@param {type:"string"}
# @markdown 「セルを実行」ボタンを押してテキストファイルを生成\
# @markdown テキストファイルはimage_folderに指定したフォルダに作成されます。


#フォルダとテキストのファイル名は自動で決める

def export_ImageFileName(textstring):
  text = ""
  for i in textstring:
    # 配列の各要素を指定した文字列を間に挟みつつ全結合
    text = "\n".join(textstring)

  # print(text)
  filepath = os.path.join(image_folder,"imageFileName_List.txt")
  with open(filepath, mode = "w") as f:
    f.write(text)
  return f"テキストファイルを{filepath}に生成"

# 関数を実行
tmp = find_image_files(image_folder)
export_ImageFileName(tmp)

In [ ]:
# @title 開発中　{display-mode: "form"}
image_dir_name = "/content/drive/MyDrive/for_mit"
image_dir_name = "/content/drive/MyDrive/Untitled Folder"
files = extract_fullfileName(image_dir_name)

for file in files:
    imga = Image.open(file)
    a = imga.height * imga.width
    print(a)